# Dataset về đề xuất nhạc trong ô tô có tính đến ngữ cảnh


### Import các thư viện cần thiết

In [308]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

# Thiết lập hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('default')
sns.set_palette('husl')

### Đọc dữ liệu từ file Excel

In [309]:
# Đọc file Excel từ nhiều sheet như trong exploration notebook
con_rat = pd.read_excel('./Data_InCarMusic.xlsx', sheet_name=0).rename(columns={' Rating': 'label', 'UserID': 'user', 'ItemID': 'item'})
con_fac = pd.read_excel('./Data_InCarMusic.xlsx', sheet_name=1)
mus_trk = pd.read_excel('./Data_InCarMusic.xlsx', sheet_name=2).rename(columns={' category_id': 'category_id'})
mus_cat = pd.read_excel('./Data_InCarMusic.xlsx', sheet_name=3, header=None).rename(columns={0:'genre_id', 1:'genre'})

# Tạo dictionary cho genre mapping
mus_cat_dict = mus_cat.set_index('genre_id').genre.str.split(' ').str[0].to_dict()
mus_trk['genre'] = mus_trk.category_id.apply(lambda x: mus_cat_dict.get(x))

# Sử dụng con_rat làm dataframe chính
df = con_rat.copy()

print(f'\nKích thước dữ liệu: {df.shape[0]} hàng x {df.shape[1]} cột')
print(f'Các cột: {list(df.columns)}')



Kích thước dữ liệu: 4012 hàng x 11 cột
Các cột: ['user', 'item', 'label', 'DrivingStyle', 'landscape', 'mood', 'naturalphenomena ', 'RoadType', 'sleepiness', 'trafficConditions', 'weather']


## Tiền xử lý dữ liệu

In [310]:
# Tìm các cột context (categorical)
cat_cols = df.columns[df.dtypes == 'object']
print("Các cột context:", list(cat_cols))

# In ra các giá trị unique cho mỗi cột context
print("\nCác giá trị unique cho mỗi cột context:")
for c in cat_cols:
    unique_vals = df[c].unique()
    print(f'{str(unique_vals):<55} #{len(unique_vals)}')

# Tạo binary label như trong exploration
df['label'] = np.select([df.label > 3, df.label <= 3], [1, 0])

# Xử lý missing values - thay thế NaN bằng 'Unknown' thay vì loại bỏ
for col in cat_cols:
    df[col] = df[col].fillna('Unknown')

print(f"\nSau khi xử lý missing values:")
print(f"Số dòng: {len(df)}")
print(f"Số user: {df['user'].nunique()}")
print(f"Số item: {df['item'].nunique()}")


Các cột context: ['DrivingStyle', 'landscape', 'mood', 'naturalphenomena ', 'RoadType', 'sleepiness', 'trafficConditions', 'weather']

Các giá trị unique cho mỗi cột context:
[nan 'relaxed driving' 'sport driving']                 #3
[nan 'urban' 'mountains' 'country side' 'coast line']   #5
[nan 'sad' 'lazy' 'active' 'happy']                     #5
[nan 'night' 'morning' 'day time' 'afternoon']          #5
[nan 'city' 'serpentine' 'highway']                     #4
[nan 'sleepy' 'awake']                                  #3
[nan 'traffic jam' 'lots of cars' 'free road']          #4
['sunny' 'snowing' 'rainy' 'cloudy' nan]                #5

Sau khi xử lý missing values:
Số dòng: 4012
Số user: 42
Số item: 139


In [311]:
# Xử lý context với one-hot encoding như trong exploration notebook
# Tạo one-hot encoding cho các cột context
features_oh = pd.get_dummies(df[cat_cols], dummy_na=True)

# Tạo context string từ one-hot encoding
context_series = (features_oh * features_oh.columns).sum(axis=1)
context_series.rename('context', inplace=True)

# Tạo dataframe cuối cùng với context
feat_data = pd.concat([df[['user', 'item', 'label']], context_series], axis=1)

print("Dataframe sau khi xử lý context:")
print(feat_data.head())
print(f"\nSố lượng context unique: {feat_data['context'].nunique()}")
print(f"Các context mẫu: {feat_data['context'].unique()[:10]}")


Dataframe sau khi xử lý context:
   user  item  label                                            context
0  1001   715      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
1  1001   267      1  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
2  1001   294      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
3  1001   259      1  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
4  1001   674      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...

Số lượng context unique: 27
Các context mẫu: ['DrivingStyle_Unknownlandscape_Unknownmood_Unknownnaturalphenomena _UnknownRoadType_Unknownsleepiness_UnknowntrafficConditions_Unknownweather_sunny'
 'DrivingStyle_Unknownlandscape_Unknownmood_Unknownnaturalphenomena _UnknownRoadType_Unknownsleepiness_UnknowntrafficConditions_Unknownweather_snowing'
 'DrivingStyle_Unknownlandscape_Unknownmood_Unknownnaturalphenomena _UnknownRoadType_Unknownsleepiness_UnknowntrafficConditions_Unknownweather_rainy'
 'DrivingStyle_Unknownlandscape_Unkno

In [312]:
# Xuất dữ liệu 
import os
out_dir = 'output_carskit'
os.makedirs(out_dir, exist_ok=True)

# Xuất ratings với context (cho LightFM)
ratings_ctx_path = os.path.join(out_dir, 'ratings_with_context.csv')
feat_data.to_csv(ratings_ctx_path, index=False, encoding='utf-8')

# Xuất ratings chỉ có user, item, label (cho collaborative filtering)
ratings_only = feat_data[['user', 'item', 'label']].copy()
ratings_only_path = os.path.join(out_dir, 'ratings_only.csv')
ratings_only.to_csv(ratings_only_path, index=False, encoding='utf-8')

# Xuất thông tin item với genre (cho item features)
item_features = mus_trk[['id', 'genre']].rename(columns={'id': 'item'})
item_features_path = os.path.join(out_dir, 'item_features.csv')
item_features.to_csv(item_features_path, index=False, encoding='utf-8')

print("Đã lưu các file:")
print(f" - {ratings_ctx_path}")
print(f" - {ratings_only_path}")
print(f" - {item_features_path}")

# Thống kê cuối cùng
print(f"\nThống kê cuối cùng:")
print(f"Số user: {feat_data['user'].nunique()}")
print(f"Số item: {feat_data['item'].nunique()}")
print(f"Số interaction: {len(feat_data)}")
print(f"Số context unique: {feat_data['context'].nunique()}")
print(f"Tỷ lệ positive (label=1): {feat_data['label'].mean():.3f}")


Đã lưu các file:
 - output_carskit\ratings_with_context.csv
 - output_carskit\ratings_only.csv
 - output_carskit\item_features.csv

Thống kê cuối cùng:
Số user: 42
Số item: 139
Số interaction: 4012
Số context unique: 27
Tỷ lệ positive (label=1): 0.258


## Train mô hình gợi ý

In [313]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from models.model_classes import BMF, NeuMF, LNCM, ENCM

DATA_DIR = r"D:\DATN-CNTT-2025-CoLien-B21DCCN433\Demo\output_carskit"
RATINGS_ONLY = os.path.join(DATA_DIR, "ratings_only.csv")
RATINGS_CTX = os.path.join(DATA_DIR, "ratings_with_context.csv")

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

df_only = pd.read_csv(RATINGS_ONLY)          # columns: user, item, label
df_ctx = pd.read_csv(RATINGS_CTX)            # columns: user, item, label, context (string)

# Binary labels already prepared in your pipeline (0/1)
df_only = df_only.astype({"user": str, "item": str, "label": int})
df_ctx = df_ctx.astype({"user": str, "item": str, "label": int, "context": str})

print(df_only.head())
print(df_ctx.head())
print("Users:", df_only["user"].nunique(), "Items:", df_only["item"].nunique(), "Interactions:", len(df_only))

   user item  label
0  1001  715      0
1  1001  267      1
2  1001  294      0
3  1001  259      1
4  1001  674      0
   user item  label                                            context
0  1001  715      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
1  1001  267      1  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
2  1001  294      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
3  1001  259      1  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
4  1001  674      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
Users: 42 Items: 139 Interactions: 4012


In [314]:
user_le = LabelEncoder().fit(df_only["user"].values)
item_le = LabelEncoder().fit(df_only["item"].values)

# Use the combined 'context' string as one categorical feature for ENCM
ctx_le = LabelEncoder().fit(df_ctx["context"].values)

df_only["u_idx"] = user_le.transform(df_only["user"])
df_only["i_idx"] = item_le.transform(df_only["item"])

df_ctx["u_idx"] = user_le.transform(df_ctx["user"])
df_ctx["i_idx"] = item_le.transform(df_ctx["item"])
df_ctx["c_idx"] = ctx_le.transform(df_ctx["context"])

n_users = df_only["u_idx"].max() + 1
n_items = df_only["i_idx"].max() + 1
n_contexts = [df_ctx["c_idx"].max() + 1]  # single context field

n_users, n_items, n_contexts

def per_user_split(df, user_col="u_idx", frac_test=0.2, frac_val=0.1, seed=SEED):
    train_rows, test_rows = [], []
    for u, grp in df.groupby(user_col):
        if len(grp) < 2:
            train_rows.append(grp)
            continue
        tr, te = train_test_split(grp, test_size=frac_test, random_state=seed, shuffle=True)
        train_rows.append(tr)
        test_rows.append(te)
    train_df = pd.concat(train_rows).reset_index(drop=True)
    test_df = pd.concat(test_rows).reset_index(drop=True) if test_rows else train_df.iloc[0:0].copy()

    # val from train
    if len(train_df) > 1:
        tr_rows, val_rows = [], []
        for u, grp in train_df.groupby(user_col):
            if len(grp) < 2:
                tr_rows.append(grp)
                continue
            tr, va = train_test_split(grp, test_size=frac_val, random_state=seed, shuffle=True)
            tr_rows.append(tr)
            val_rows.append(va)
        train_df = pd.concat(tr_rows).reset_index(drop=True)
        val_df = pd.concat(val_rows).reset_index(drop=True) if val_rows else train_df.iloc[0:0].copy()
    else:
        val_df = train_df.iloc[0:0].copy()

    return train_df, val_df, test_df

train_only, val_only, test_only = per_user_split(df_only)
train_ctx,  val_ctx,  test_ctx  = per_user_split(df_ctx)

len(train_only), len(val_only), len(test_only), len(train_ctx), len(val_ctx), len(test_ctx)

(2863, 341, 808, 2863, 341, 808)

In [315]:
def build_user_positives(df, user_col="u_idx", item_col="i_idx", label_col="label"):
    pos = defaultdict(set)
    for u, it, y in df[[user_col, item_col, label_col]].itertuples(index=False):
        if y == 1:
            pos[u].add(it)
    return pos

train_pos = build_user_positives(train_only)
all_items = np.arange(n_items, dtype=np.int32)

# Increase negatives for harder training
DEF_NUM_NEG = 10

def make_pairs(df, user_col="u_idx", item_col="i_idx", label_col="label", num_neg=DEF_NUM_NEG):
    users, items, labels = [], [], []
    by_user = df.groupby(user_col)
    for u, grp in by_user:
        pos_items = set(grp.loc[grp[label_col] == 1, item_col].tolist())
        if not pos_items:
            continue
        for it in pos_items:
            users.append(u); items.append(it); labels.append(1)
            # Negatives
            neg_count = 0
            while neg_count < num_neg:
                j = np.random.randint(0, n_items)
                if (j not in pos_items):
                    users.append(u); items.append(j); labels.append(0)
                    neg_count += 1
    return np.array(users, dtype=np.int32), np.array(items, dtype=np.int32), np.array(labels, dtype=np.float32)

tr_u, tr_i, tr_y = make_pairs(train_only)
va_u, va_i, va_y = make_pairs(val_only) if len(val_only) else (tr_u[:0], tr_i[:0], tr_y[:0])

In [316]:
# Tạo tf.data.Dataset cho mô hình 2-input (BMF/NeuMF/LNCM) và 3-input (ENCM)
import tensorflow as tf
import numpy as np

BATCH_SIZE = 256
SHUFFLE_BUF = 100_000

# 2-input datasets: (user_idx, item_idx) -> label
train_ds_2 = tf.data.Dataset.from_tensor_slices(((tr_u, tr_i), tr_y)) \
    .shuffle(min(len(tr_y), SHUFFLE_BUF), seed=SEED, reshuffle_each_iteration=True) \
    .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds_2 = tf.data.Dataset.from_tensor_slices(((va_u, va_i), va_y)) \
    .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# 3-input datasets cho ENCM: (user_idx, item_idx, context_idx[None]) -> label
# Tạo cặp có ngữ cảnh từ tập đã có c_idx (train_ctx/val_ctx)
from collections import defaultdict

def make_pairs_with_ctx(df, user_col="u_idx", item_col="i_idx", ctx_col="c_idx", label_col="label", num_neg=DEF_NUM_NEG):
    users, items, ctxs, labels = [], [], [], []
    by_user = df.groupby(user_col)
    for u, grp in by_user:
        pos_rows = grp.loc[grp[label_col] == 1, [item_col, ctx_col]]
        pos_items = set(pos_rows[item_col].tolist())
        if not len(pos_rows):
            continue
        for it, c in pos_rows.itertuples(index=False):
            # positive
            users.append(u); items.append(it); ctxs.append(c); labels.append(1.0)
            # negatives với cùng context của positive để giữ ngữ cảnh ổn định
            neg_count = 0
            while neg_count < num_neg:
                j = np.random.randint(0, n_items)
                if j not in pos_items:
                    users.append(u); items.append(j); ctxs.append(c); labels.append(0.0)
                    neg_count += 1
    return (
        np.array(users, dtype=np.int32),
        np.array(items, dtype=np.int32),
        np.array(ctxs, dtype=np.int32),
        np.array(labels, dtype=np.float32),
    )

tr_u3, tr_i3, tr_c3, tr_y3 = make_pairs_with_ctx(train_ctx)
va_u3, va_i3, va_c3, va_y3 = make_pairs_with_ctx(val_ctx) if len(val_ctx) else (tr_u3[:0], tr_i3[:0], tr_c3[:0], tr_y3[:0])

# reshape context sang (N, 1) như ENCM.build([(None,), (None,), (None, 1)])
tr_c3_inp = tr_c3.reshape(-1, 1)
va_c3_inp = va_c3.reshape(-1, 1)

train_ds_3 = tf.data.Dataset.from_tensor_slices(((tr_u3, tr_i3, tr_c3_inp), tr_y3)) \
    .shuffle(min(len(tr_y3), SHUFFLE_BUF), seed=SEED, reshuffle_each_iteration=True) \
    .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds_3 = tf.data.Dataset.from_tensor_slices(((va_u3, va_i3, va_c3_inp), va_y3)) \
    .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Lưu train_encm để sử dụng ở phần builder đánh giá top-K (lấy mode context per-user)
train_encm = train_ctx[["u_idx", "i_idx", "c_idx", "label"]].copy()

In [317]:
def compile_binary(model, lr=1e-3):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=[tf.keras.metrics.AUC(name="auc"), tf.keras.metrics.BinaryAccuracy(name="acc")]
    )

def fit_model(model, train_ds, val_ds, epochs=5):
    cb = [
        tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=2, restore_best_weights=True)
    ]
    hist = model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=1, callbacks=cb)
    return hist

In [318]:
bmf = BMF(n_users=n_users, n_items=n_items, embedding_dim=50)
# build shapes
bmf.build([(None,), (None,)])
compile_binary(bmf, lr=1e-3)
fit_model(bmf, train_ds_2, val_ds_2, epochs=15)

Epoch 1/15


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'bmf_17', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.7605 - auc: 0.5109 - loss: 0.6744 - val_acc: 0.8792 - val_auc: 0.5246 - val_loss: 0.6530
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9002 - auc: 0.5715 - loss: 0.6387 - val_acc: 0.9091 - val_auc: 0.5393 - val_loss: 0.6199
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.5959 - loss: 0.6029 - val_acc: 0.9091 - val_auc: 0.5538 - val_loss: 0.5850
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.6114 - loss: 0.5625 - val_acc: 0.9091 - val_auc: 0.5642 - val_loss: 0.5448
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.6436 - loss: 0.5140 - val_acc: 0.9091 - val_auc: 0.5760 - val_loss: 0.4996
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.6624 - loss: 0.4608 - val_acc: 0.9091 - val_auc: 0.5861 - val_loss: 0.4527
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.6826 - loss: 0.4091 - val_acc: 0.9091 - val_auc: 

In [319]:
neumf = NeuMF(n_users=n_users, n_items=n_items, embedding_dim=50, hidden_dims=[64,32,16])
neumf.build([(None,), (None,)])
compile_binary(neumf, lr=1e-3)
fit_model(neumf, train_ds_2, val_ds_2, epochs=15)

Epoch 1/15


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'neu_mf_14', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - acc: 0.8588 - auc: 0.4805 - loss: 0.6745 - val_acc: 0.9091 - val_auc: 0.5135 - val_loss: 0.6380
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.5192 - loss: 0.5774 - val_acc: 0.9091 - val_auc: 0.5140 - val_loss: 0.4709
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.5136 - loss: 0.3878 - val_acc: 0.9091 - val_auc: 0.5253 - val_loss: 0.3188
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.5222 - loss: 0.3354 - val_acc: 0.9091 - val_auc: 0.5524 - val_loss: 0.3109
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.5948 - loss: 0.3097 - val_acc: 0.9091 - val_auc: 0.5828 - val_loss: 0.3044
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.6609 - loss: 0.2950 - val_acc: 0.9091 - val_auc: 0.5979 - val_loss: 0.3029
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.6652 - loss: 0.2933 - val_acc: 0.9091 - val_auc: 

In [320]:
lncm = LNCM(n_users=n_users, n_items=n_items, embedding_dim=50, hidden_dims=[64,32])
lncm.build([(None,), (None,)])
compile_binary(lncm, lr=1e-3)
fit_model(lncm, train_ds_2, val_ds_2, epochs=15)

Epoch 1/15


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'lncm_14', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - acc: 0.9091 - auc: 0.5098 - loss: 0.3376 - val_acc: 0.9091 - val_auc: 0.5421 - val_loss: 0.3083
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.6343 - loss: 0.2940 - val_acc: 0.9091 - val_auc: 0.5901 - val_loss: 0.2988
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.7008 - loss: 0.2815 - val_acc: 0.9091 - val_auc: 0.5981 - val_loss: 0.3153
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.7171 - loss: 0.2780 - val_acc: 0.9091 - val_auc: 0.6054 - val_loss: 0.3151
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.7205 - loss: 0.2762 - val_acc: 0.9091 - val_auc: 0.6073 - val_loss: 0.3154
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.7225 - loss: 0.2760 - val_acc: 0.9091 - val_auc: 0.6145 - val_loss: 0.3149
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.9091 - auc: 0.7283 - loss: 0.2744 - val_acc: 0.9091 - val_auc: 

In [321]:
encm = ENCM(n_users=n_users, n_items=n_items, n_contexts=n_contexts, embedding_dim=50, context_dim=10, hidden_dims=[64,32])
# Inputs: user_ids, item_ids, context_features (shape: (batch, 1))
encm.build([(None,), (None,), (None, 1)])
compile_binary(encm, lr=1e-3)
fit_model(encm, train_ds_3, val_ds_3, epochs=15)

Epoch 1/15


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'encm_14', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - acc: 0.8876 - auc: 0.5128 - loss: 0.5355 - val_acc: 0.9091 - val_auc: 0.5400 - val_loss: 0.3286
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.5829 - loss: 0.3105 - val_acc: 0.9091 - val_auc: 0.6272 - val_loss: 0.2978
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.7063 - loss: 0.2829 - val_acc: 0.9091 - val_auc: 0.6385 - val_loss: 0.3004
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.7387 - loss: 0.2732 - val_acc: 0.9091 - val_auc: 0.6385 - val_loss: 0.3093
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.7431 - loss: 0.2715 - val_acc: 0.9091 - val_auc: 0.6379 - val_loss: 0.3099
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9091 - auc: 0.7572 - loss: 0.2659 - val_acc: 0.9091 - val_auc: 0.6392 - val_loss: 0.3081
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9096 - auc: 0.7683 - loss: 0.2620 - val_acc: 0.9103 - val_auc: 0

In [322]:
# Leave-One-Out style evaluation (99 negatives per test positive), with correct context for ENCM
import numpy as np
import pandas as pd
from collections import defaultdict

# Build dictionaries of user->seen items across splits to avoid leakage
user_seen = defaultdict(set)
for df_ in [train_only, val_only, test_only]:
    for u, it, y in df_[["u_idx","i_idx","label"]].itertuples(index=False):
        if y == 1:
            user_seen[u].add(it)

# Test positives as interaction list
test_pos_rows = test_only.loc[test_only["label"] == 1, ["u_idx","i_idx"]].copy()
# For ENCM, map to context id from test_ctx
test_ctx_map = {}
if 'test_ctx' in globals() and len(test_ctx):
    # build mapping (u,i)->c; if multiple rows exist, use the most frequent
    tmp = test_ctx.groupby(["u_idx","i_idx"])['c_idx'].agg(lambda s: s.value_counts().index[0])
    test_ctx_map = tmp.to_dict()

rng = np.random.default_rng(SEED)


def evaluate_loo(model, builder, k=10, num_neg=99):
    hits, ndcgs, precs, maps = [], [], [], []

    def dcg_at_k(rank):
        return 1.0 / np.log2(rank + 2)  # rank is 0-indexed

    for u, i_pos in test_pos_rows[["u_idx","i_idx"]].itertuples(index=False):
        seen = user_seen.get(u, set()) - {i_pos}
        # sample negatives that are not seen
        neg_pool = [j for j in range(n_items) if j not in seen and j != i_pos]
        if len(neg_pool) < num_neg:
            # fallback: use whatever available
            sampled_negs = neg_pool
        else:
            sampled_negs = rng.choice(neg_pool, size=num_neg, replace=False).tolist()

        candidates = np.array([i_pos] + sampled_negs, dtype=np.int32)
        batch_user = np.full_like(candidates, u, dtype=np.int32)

        # context for ENCM: use exact test context if available, else per-user mode, else 0
        c_id = int(test_ctx_map.get((int(u), int(i_pos)),  
                 user_ctx_mode.get(int(u), 0)))
        model_inputs = builder(batch_user, candidates, u, c_id)

        scores = model(model_inputs, training=False).numpy().reshape(-1)
        top_idx = np.argsort(-scores)[:k]
        top_items = candidates[top_idx].tolist()

        # metrics
        hit = 1.0 if i_pos in top_items else 0.0
        hits.append(hit)

        # nDCG@k
        if hit:
            rank = top_items.index(i_pos)  # 0-indexed
            ndcgs.append(dcg_at_k(rank))
        else:
            ndcgs.append(0.0)

        # Precision@k (either 0 or 1/k because only one positive)
        precs.append(1.0/ k if hit else 0.0)

        # MAP@k equals precision at rank of the hit when it occurs
        if hit:
            r = top_items.index(i_pos) + 1
            maps.append(1.0 / r)
        else:
            maps.append(0.0)

    return {
        'HR@{}'.format(k): float(np.mean(hits)) if hits else 0.0,
        'nDCG@{}'.format(k): float(np.mean(ndcgs)) if ndcgs else 0.0,
        'Precision@{}'.format(k): float(np.mean(precs)) if precs else 0.0,
        'MAP@{}'.format(k): float(np.mean(maps)) if maps else 0.0,
    }

# Builders updated to accept explicit context id for ENCM
def two_inp_builder_eval(u_arr, i_arr, u_single, c_id_unused):
    return (u_arr, i_arr)

def three_inp_builder_eval(u_arr, i_arr, u_single, c_id):
    c_arr = np.full((len(u_arr), 1), c_id, dtype=np.int32)
    return (u_arr, i_arr, c_arr)

models_eval = {
    'BMF': bmf,
    'NeuMF': neumf,
    'LNCM': lncm,
    'ENCM': encm,
}

builders_eval = {
    'BMF': two_inp_builder_eval,
    'NeuMF': two_inp_builder_eval,
    'LNCM': two_inp_builder_eval,
    'ENCM': three_inp_builder_eval,
}

rows = []
for name, model in models_eval.items():
    metrics = evaluate_loo(model, builders_eval[name], k=10, num_neg=99)
    row = {'model': name}
    row.update(metrics)
    rows.append(row)

results_loo = pd.DataFrame(rows)
print(results_loo.to_string(index=False))

model    HR@10  nDCG@10  Precision@10   MAP@10
  BMF 0.502370 0.413544      0.050237 0.386371
NeuMF 0.497630 0.377705      0.049763 0.340038
 LNCM 0.336493 0.177495      0.033649 0.128987
 ENCM 0.502370 0.421782      0.050237 0.397418
